<a href="https://colab.research.google.com/github/psa2001/ml1.sphere/blob/master/pyatkovsky_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Общая информация**

**Срок сдачи:** 29 марта 2021, 08:30 

**Штраф за опоздание:** по 1 баллу за 24 часа задержки. Через 5 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0220, Задание 1] Пятковский Сергей.

Используйте данный Ipython Notebook при оформлении домашнего задания.

In [21]:
!git config --global user.email "pyatkovsky15022001@gmail.com"
!git config --global user.name "psa2001"


In [22]:
!echo "# ml_technosphera_hw" >> README.md
!git init
!git add README.md
!git commit -m "first commit"
!git branch -M main
!git remote add origin https://github.com/psa2001/ml_technosphera_hw.git
!git push -u origin main


Reinitialized existing Git repository in /content/.git/
[master (root-commit) 770d8d9] first commit
 1 file changed, 2 insertions(+)
 create mode 100644 README.md
fatal: remote origin already exists.
fatal: could not read Username for 'https://github.com': No such device or address



**Штрафные баллы:**

1. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
2. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [1]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import BaseEstimator
from sklearn.datasets import fetch_20newsgroups
#from sklearn.datasets import fetch_mldata

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. 
Для подсчета расстояний можно использовать функции [отсюда](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

In [2]:
from sklearn.neighbors import KDTree

class MyKNeighborsClassifier(BaseEstimator):
    
    def __init__(self, n_neighbors, algorithm='brute'):
        self.n_neighbors = n_neighbors
        self.algorithm = algorithm
        self.points = []
        self.classes = []
        self.p = 2 #minkowski with p = 2
        self.leaf_size = 30
        pass
    
    def fit(self, X, y):
        self.points = X
        self.classes = y
        pass
    
    def predict(self, X):
        if(self.algorithm == 'brute'):
            X1 = np.copy(X)
            for i in range(self.points.shape[0] - 1):
                X1 = np.concatenate([X1, X])
            fut_shape = (X.shape)
            fut_shape = np.insert(fut_shape, 0, self.points.shape[0])
            X1 = X1.reshape(fut_shape)

            points1 = np.repeat(self.points, X.shape[0], axis = 0)
            points1 = points1.reshape(X1.shape)
            
            minkowski_res = np.linalg.norm((X1 - points1), ord = 2, axis = 2)
            #
            
            most_close = (self.classes[np.argpartition(minkowski_res, 2, axis = 0)[:2]]).T
            answ = np.array([], dtype = np.int32)
            for most_pop in most_close:
                answ = np.append(answ, np.argmax(np.bincount(most_pop)))

        if(self.algorithm == 'kd_tree'):
            tree = KDTree(self.points, self.leaf_size)
            dist, ind = tree.query(X, self.n_neighbors)
            most_close = self.classes[ind]
            answ = np.array([], dtype = np.int32)
            for most_pop in most_close:
                answ = np.append(answ, np.argmax(np.bincount(most_pop)))
        return answ
    

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [3]:
iris = datasets.load_iris()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [5]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [6]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [7]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. 

In [8]:
%time clf.fit(X_train, y_train)

CPU times: user 837 µs, sys: 0 ns, total: 837 µs
Wall time: 979 µs


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [9]:
%time my_clf.fit(X_train, y_train)

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 12.9 µs


In [10]:
%time clf.predict(X_test)

CPU times: user 2.3 ms, sys: 81 µs, total: 2.38 ms
Wall time: 3.23 ms


array([2, 2, 0, 2, 1, 1, 0, 0, 0, 1, 1, 1, 2, 2, 0])

In [11]:
%time my_clf.predict(X_test)

CPU times: user 1.49 ms, sys: 0 ns, total: 1.49 ms
Wall time: 1.74 ms


array([2, 2, 0, 2, 1, 1, 0, 0, 0, 1, 1, 1, 2, 2, 0])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. Точность не должна уступать значению KNN из sklearn. 

In [12]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [13]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [14]:
%time clf.fit(X_train, y_train)

CPU times: user 1.55 ms, sys: 171 µs, total: 1.72 ms
Wall time: 5.41 ms


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [15]:
%time my_clf.fit(X_train, y_train)

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 9.3 µs


In [16]:
%time clf.predict(X_test)

CPU times: user 1.15 ms, sys: 1.04 ms, total: 2.19 ms
Wall time: 2.22 ms


array([2, 0, 1, 1, 0, 0, 1, 1, 0, 2, 0, 2, 1, 2, 1])

In [17]:
%time my_clf.predict(X_test)

CPU times: user 1.79 ms, sys: 205 µs, total: 2 ms
Wall time: 2.42 ms


array([2, 0, 1, 1, 0, 0, 1, 1, 0, 2, 0, 2, 1, 2, 1])

In [18]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [ ]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [ ]:
data = newsgroups['data']
target = newsgroups['target']

Преобразуйте текстовые данные из data с помощью [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Словарь можно ограничить по частотности.

*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальную точность в среднем на валидации на 3 фолдах).
Постройте график зависимости средней точности от количества соседей. Можно рассмотреть число соседей от 1 до 10.

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf векторизацию( sklearn.feature_extraction.text.TfidfVectorizer)

Сравните модели, выберите лучшую.

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [ ]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим точность вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.